In [ ]:
import pandas as pd
import collections, numpy,os
from collections import Counter
import numpy as np
import text_normalization
from sklearn.preprocessing import LabelEncoder
import re

In [ ]:
df = pd.read_excel('../data/Amal Twitts.xlsx')

In [ ]:
exts = df['text'].to_list()

In [ ]:
df['norm_text'] = df.text.apply(lambda text: text_normalization.clean_unicode(str(text)))

In [ ]:
df

,author_id,username,author_followers,author_tweets,author_description,author_location,tweet_id,text,created_at,retweets,replies,likes,norm_text
0,805528640184287238,raumunz,3070,1054648,LIC. RAÚL MUÑOZ ALVAREZ. ABOGADO/NOTARIO. DERE...,"San José, Costa Rica",1.547008e+18,الحج المبرور ليس له جزاء إلا الجنة#السيرة_كأن...,2022-07-12 23:59:55+00:00,222.0,0.0,0.0,الحج المبرور ليس له جزاء الا الجنه السيره كان...
1,344948845,hesheey,713,19244,هلالي موت --وطني 🇸🇦 فخري وعزتي -- قاعدتي في ال...,NaN,1.547008e+18,رئيس ساحل العاج يرفض البروتوكول السعودي في الحج,2022-07-12 23:59:43+00:00,161.0,0.0,0.0,رئيس ساحل العاج يرفض البروتوكول السعودى فى الحج
2,1281817306046365697,ealam_itti,22300,22243,#عالم_الاتحاد منصه إخباريه وتفاعلية لتغطيه أخب...,NaN,1.547008e+18,ما تقوم به بلادنا في خدمة حجاج بيت الله الحرا...,2022-07-12 23:59:42+00:00,692.0,0.0,0.0,ما تقوم به بلادنا فى خدمه حجاج بيت الله الحرا...
3,1442923975223414786,3krstzzm,54,7054,"‏""أكرمتني فلك الحمد، وسترتني فلك الحمد، ورزقتن...","الرياض, المملكة العربية السعود",1.547008e+18,شكرا_قيادتنا_على_نجاح_الحج الحمدلله حتى يبلغ ا...,2022-07-12 23:59:37+00:00,2681.0,0.0,0.0,شكرا قيادتنا على نجاح الحج الحمدلله حتى يبلغ ا...
4,417380510,othman111mo,3563,146035,مهتم بكل شئ وخاصه كل ما يدور في وطني,Kingdom of Saudi Arabia,1.547008e+18,الاتحاد الأوروبي: السعودية نجحت بإدارة موسم #...,2022-07-12 23:59:35+00:00,63.0,0.0,0.0,الاتحاد الاوروبى السعوديه نجحت باداره موسم ال...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140644,1510675396987084804,abwmhmd42014503,21,3471,حساب جديد,NaN,1.544834e+18,@marsdnews24 الله يقويهم على المفسدين اللي فتر...,2022-07-07 00:00:17+00:00,0.0,0.0,0.0,الله يقويهم على المفسدين اللى فتره الحج بالنس...
140645,790485202481123328,Shreefayahya,49,3695,NaN,NaN,1.544834e+18,أمثل هذا يؤتمن على شعيرة الحج؟!\nاللهم إنا نبر...,2022-07-07 00:00:03+00:00,1.0,1.0,2.0,امثل هذا يتمن على شعيره الحج اللهم انا نبرا ال...
140646,1321126124869603329,AlarabiyaJordan,3695,8643,"""العربية الأردن"" حساب متخصص في تغطية كافة جوان...",NaN,1.544834e+18,أكد وزير الاوقاف والشؤون والمقدسات الاسلامية م...,2022-07-07 00:00:02+00:00,0.0,0.0,3.0,اكد وزير الاوقاف والشون والمقدسات الاسلاميه مح...
140647,216289357,skynewsarabia,8047144,348894,NaN,"Abu Dhabi, UAE",1.544834e+18,قطار المشاعر المقدسة.. رحلة لدقائق بدل ساعات |...,2022-07-07 00:00:00+00:00,10.0,0.0,26.0,قطار المشاعر المقدسه رحله لدقائق بدل ساعات غرف...


In [ ]:
# to remove retweeted tweets as they cannot be lnger than 140 chars and usulally contain repeated info 
match = 'RT'
print('Number of Tweets before removing rows contain unwnated word: ',df.shape)
df = df[df['text'].str.contains(match)==False]
print('Number of Tweets after removing rows contain unwnated word: ',df.shape)

Number of Tweets before removing rows contain unwnated word:  (140649, 13)
Number of Tweets after removing rows contain unwnated word:  (109293, 13)


In [ ]:
# to remove mssing valuse
print('Number of Tweets before removing mssing valuse: ',df.shape)
df = df.dropna(subset=['norm_text'])  
print('Number of Tweets after removing mssing valuse: ',df.shape)
df.head(5)

Number of Tweets before removing mssing valuse:  (109293, 13)
Number of Tweets after removing mssing valuse:  (109293, 13)


,author_id,username,author_followers,author_tweets,author_description,author_location,tweet_id,text,created_at,retweets,replies,likes,norm_text
0,805528640184287238,raumunz,3070,1054648,LIC. RAÚL MUÑOZ ALVAREZ. ABOGADO/NOTARIO. DERE...,"San José, Costa Rica",1.547008e+18,الحج المبرور ليس له جزاء إلا الجنة#السيرة_كأن...,2022-07-12 23:59:55+00:00,222.0,0.0,0.0,الحج المبرور ليس له جزاء الا الجنه السيره كان...
1,344948845,hesheey,713,19244,هلالي موت --وطني 🇸🇦 فخري وعزتي -- قاعدتي في ال...,NaN,1.547008e+18,رئيس ساحل العاج يرفض البروتوكول السعودي في الحج,2022-07-12 23:59:43+00:00,161.0,0.0,0.0,رئيس ساحل العاج يرفض البروتوكول السعودى فى الحج
2,1281817306046365697,ealam_itti,22300,22243,#عالم_الاتحاد منصه إخباريه وتفاعلية لتغطيه أخب...,NaN,1.547008e+18,ما تقوم به بلادنا في خدمة حجاج بيت الله الحرا...,2022-07-12 23:59:42+00:00,692.0,0.0,0.0,ما تقوم به بلادنا فى خدمه حجاج بيت الله الحرا...
3,1442923975223414786,3krstzzm,54,7054,"‏""أكرمتني فلك الحمد، وسترتني فلك الحمد، ورزقتن...","الرياض, المملكة العربية السعود",1.547008e+18,شكرا_قيادتنا_على_نجاح_الحج الحمدلله حتى يبلغ ا...,2022-07-12 23:59:37+00:00,2681.0,0.0,0.0,شكرا قيادتنا على نجاح الحج الحمدلله حتى يبلغ ا...
4,417380510,othman111mo,3563,146035,مهتم بكل شئ وخاصه كل ما يدور في وطني,Kingdom of Saudi Arabia,1.547008e+18,الاتحاد الأوروبي: السعودية نجحت بإدارة موسم #...,2022-07-12 23:59:35+00:00,63.0,0.0,0.0,الاتحاد الاوروبى السعوديه نجحت باداره موسم ال...


In [ ]:
# to remove duplicates from clean text column
print('Number of Tweets before removing duplicates: ',df.shape)
df = df.drop_duplicates(['norm_text'])
print('Number of Tweets after removing duplicates: ',df.shape)
df.head(5)

Number of Tweets before removing duplicates:  (109293, 13)
Number of Tweets after removing duplicates:  (86413, 13)


,author_id,username,author_followers,author_tweets,author_description,author_location,tweet_id,text,created_at,retweets,replies,likes,norm_text
0,805528640184287238,raumunz,3070,1054648,LIC. RAÚL MUÑOZ ALVAREZ. ABOGADO/NOTARIO. DERE...,"San José, Costa Rica",1.547008e+18,الحج المبرور ليس له جزاء إلا الجنة#السيرة_كأن...,2022-07-12 23:59:55+00:00,222.0,0.0,0.0,الحج المبرور ليس له جزاء الا الجنه السيره كان...
1,344948845,hesheey,713,19244,هلالي موت --وطني 🇸🇦 فخري وعزتي -- قاعدتي في ال...,NaN,1.547008e+18,رئيس ساحل العاج يرفض البروتوكول السعودي في الحج,2022-07-12 23:59:43+00:00,161.0,0.0,0.0,رئيس ساحل العاج يرفض البروتوكول السعودى فى الحج
2,1281817306046365697,ealam_itti,22300,22243,#عالم_الاتحاد منصه إخباريه وتفاعلية لتغطيه أخب...,NaN,1.547008e+18,ما تقوم به بلادنا في خدمة حجاج بيت الله الحرا...,2022-07-12 23:59:42+00:00,692.0,0.0,0.0,ما تقوم به بلادنا فى خدمه حجاج بيت الله الحرا...
3,1442923975223414786,3krstzzm,54,7054,"‏""أكرمتني فلك الحمد، وسترتني فلك الحمد، ورزقتن...","الرياض, المملكة العربية السعود",1.547008e+18,شكرا_قيادتنا_على_نجاح_الحج الحمدلله حتى يبلغ ا...,2022-07-12 23:59:37+00:00,2681.0,0.0,0.0,شكرا قيادتنا على نجاح الحج الحمدلله حتى يبلغ ا...
4,417380510,othman111mo,3563,146035,مهتم بكل شئ وخاصه كل ما يدور في وطني,Kingdom of Saudi Arabia,1.547008e+18,الاتحاد الأوروبي: السعودية نجحت بإدارة موسم #...,2022-07-12 23:59:35+00:00,63.0,0.0,0.0,الاتحاد الاوروبى السعوديه نجحت باداره موسم ال...


In [ ]:
# function to find URL
def ck_url(x):
  re_equ = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
  ck_url = re.findall(re_equ, x)
  if ck_url:
      return True
  else:
      return False  

In [ ]:
# to remove tweets that contain URL
print('Number of Tweets before removing URLs: ',df.shape)
df = df[df['text'].apply(ck_url) == False]
print('Number of Tweets after removing URLs: ',df.shape)
df.head(5)

Number of Tweets before removing URLs:  (86413, 13)
Number of Tweets after removing URLs:  (47652, 13)


,author_id,username,author_followers,author_tweets,author_description,author_location,tweet_id,text,created_at,retweets,replies,likes,norm_text
0,805528640184287238,raumunz,3070,1054648,LIC. RAÚL MUÑOZ ALVAREZ. ABOGADO/NOTARIO. DERE...,"San José, Costa Rica",1.547008e+18,الحج المبرور ليس له جزاء إلا الجنة#السيرة_كأن...,2022-07-12 23:59:55+00:00,222.0,0.0,0.0,الحج المبرور ليس له جزاء الا الجنه السيره كان...
1,344948845,hesheey,713,19244,هلالي موت --وطني 🇸🇦 فخري وعزتي -- قاعدتي في ال...,NaN,1.547008e+18,رئيس ساحل العاج يرفض البروتوكول السعودي في الحج,2022-07-12 23:59:43+00:00,161.0,0.0,0.0,رئيس ساحل العاج يرفض البروتوكول السعودى فى الحج
2,1281817306046365697,ealam_itti,22300,22243,#عالم_الاتحاد منصه إخباريه وتفاعلية لتغطيه أخب...,NaN,1.547008e+18,ما تقوم به بلادنا في خدمة حجاج بيت الله الحرا...,2022-07-12 23:59:42+00:00,692.0,0.0,0.0,ما تقوم به بلادنا فى خدمه حجاج بيت الله الحرا...
3,1442923975223414786,3krstzzm,54,7054,"‏""أكرمتني فلك الحمد، وسترتني فلك الحمد، ورزقتن...","الرياض, المملكة العربية السعود",1.547008e+18,شكرا_قيادتنا_على_نجاح_الحج الحمدلله حتى يبلغ ا...,2022-07-12 23:59:37+00:00,2681.0,0.0,0.0,شكرا قيادتنا على نجاح الحج الحمدلله حتى يبلغ ا...
4,417380510,othman111mo,3563,146035,مهتم بكل شئ وخاصه كل ما يدور في وطني,Kingdom of Saudi Arabia,1.547008e+18,الاتحاد الأوروبي: السعودية نجحت بإدارة موسم #...,2022-07-12 23:59:35+00:00,63.0,0.0,0.0,الاتحاد الاوروبى السعوديه نجحت باداره موسم ال...


In [ ]:
columns_to_drop = ['created_at','tweet_id','author_id','author_description','likes','replies','author_location']
df = df.drop(columns=columns_to_drop)

In [ ]:
df.dropna()

,username,author_followers,author_tweets,text,retweets,norm_text
0,raumunz,3070,1054648,الحج المبرور ليس له جزاء إلا الجنة#السيرة_كأن...,222.0,الحج المبرور ليس له جزاء الا الجنه السيره كان...
1,hesheey,713,19244,رئيس ساحل العاج يرفض البروتوكول السعودي في الحج,161.0,رئيس ساحل العاج يرفض البروتوكول السعودى فى الحج
2,ealam_itti,22300,22243,ما تقوم به بلادنا في خدمة حجاج بيت الله الحرا...,692.0,ما تقوم به بلادنا فى خدمه حجاج بيت الله الحرا...
3,3krstzzm,54,7054,شكرا_قيادتنا_على_نجاح_الحج الحمدلله حتى يبلغ ا...,2681.0,شكرا قيادتنا على نجاح الحج الحمدلله حتى يبلغ ا...
4,othman111mo,3563,146035,الاتحاد الأوروبي: السعودية نجحت بإدارة موسم #...,63.0,الاتحاد الاوروبى السعوديه نجحت باداره موسم ال...
...,...,...,...,...,...,...
140631,cxi_52,7,128,@MtjrRyf @n_2ad ادخل على وازرة الحج واحجز عالذ...,0.0,ادخل على وازره الحج واحجز عالذاكرين
140632,coachshreif3ouf,42009,114342,اليوم 8 ذي الحجة هو يوم التروية حيث تبدأ اول م...,21.0,اليوم ذى الحجه هو يوم الترويه حيث تبدا اول منا...
140638,LAljmazy,15,177,@kamalsinai كتب الله لنا ولكم الحج والعمرة مرة...,0.0,كتب الله لنا ولكم الحج والعمره مره بعد مره
140640,vilnca,158,2885,يازييييين مشاعر الحج 🥹🤍,0.0,يازيين مشاعر الحج


In [ ]:
# to remove tweets that contain URL
match = 'الحوثي'
print('Number of Tweets before removing rows contain unwnated word: ',df.shape)
df = df[df['text'].str.contains(match)==False]
print('Number of Tweets after removing rows contain unwnated word: ',df.shape)

In [ ]:
df.to_csv('cleaned_data')

**To predict sentiment analsis based on MAZAJAK model**

In [ ]:
import requests
import json

In [ ]:
'''
This function offers the ability to predict the sentiment of a single sentence 
through the API, the sentiment is one of three classes (positive negative, neutral)
Input: 
        sentence(str): the input sentence of which the sentiment is to be predicted
Output:
        prediction(str): the sentiment of the given sentence 
'''


def predict(sentence):
    url = "http://mazajak.inf.ed.ac.uk:8000/api/predict"
    to_sent = {'data': sentence}
    data = json.dumps(to_sent,ensure_ascii=False).encode('utf8')
    headers = {'content-type': 'application/json'}
    # sending get request and saving the response as response object
    response = requests.post(url=url, data=data, headers=headers)

    prediction = json.loads(response.content)['data']

    return prediction


'''
This function offers the ability to predict the sentiment of a list of sentences
through the API, the sentiment is one of three classes (positive negative, neutral)
Input: 
        sent_lst(list of str): the input list of which the sentiment of its sentences is to be predicted
Output:
        prediction(list of str): the sentiments of the given sentences
'''


def predict_list(sent_lst):
    url = "http://mazajak.inf.ed.ac.uk:8000/api/predict_list"
    to_sent = {'data': sent_lst}
    data = json.dumps(to_sent)
    headers = {'content-type': 'application/json'}
    # sending get request and saving the response as response object
    response = requests.post(url=url, data=data, headers=headers)

    prediction = json.loads(response.content)['data']

    return prediction


In [ ]:
tweets = df['norm_text'].to_list()
labels = []
for tweet in tweets[:10000]:
    labels.append(predict(tweet))

In [1]:
df['auto_lable'][:10000] = labels

In [ ]:
df["auto_lable"].value_counts()

In [ ]:
df.to_csv('Lable1_data')